# پروژه دانشگاهی: تحلیل داده‌ها با استفاده از درخت تصمیم در دیتاست Iris

در این پروژه، ما با استفاده از الگوریتم درخت تصمیم داده‌های مجموعه‌ی Iris را تحلیل می‌کنیم. این پروژه شامل بارگذاری داده‌ها، ساخت و ارزیابی مدل‌های درخت تصمیم با معیارهای اطلاعاتی مختلف و تنظیم پارامترهای مدل برای بهینه‌سازی است. هدف از این پروژه بهبود دقت مدل با استفاده از روش‌های Hold-out و Cross-Validation است.

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns


## ۱. بارگذاری مجموعه داده

در این بخش از دیتاست معروف Iris استفاده می‌کنیم که شامل ۱۵۰ نمونه از سه گونه گل با ویژگی‌های مختلف است. در این مجموعه داده، هدف ما تشخیص گونه‌های مختلف بر اساس ویژگی‌های طول و عرض کاسبرگ و گلبرگ است.

In [ ]:

# بارگذاری داده‌ها و جداسازی ویژگی‌ها از متغیر هدف
data = sns.load_dataset("iris")  # مثال با دیتاست Iris
X = data.drop(columns="species")  # ویژگی‌ها
y = data["species"]               # متغیر هدف


## ۲. تقسیم‌بندی داده‌ها به بخش‌های آموزشی و آزمون

با استفاده از روش Hold-out، داده‌ها را به دو بخش آموزشی (۸۰٪) و آزمون (۲۰٪) تقسیم می‌کنیم.

In [ ]:

# تقسیم‌بندی داده‌ها به بخش‌های آموزشی و آزمون
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## ۳. ایجاد مدل درخت تصمیم با معیارهای مختلف

در این بخش، دو مدل درخت تصمیم با معیارهای `Gini` و `Information Gain` (معیار `entropy`) ایجاد می‌کنیم تا تفاوت عملکرد آنها را بررسی کنیم.

In [ ]:

# ایجاد مدل درخت تصمیم با معیارهای Gini و Entropy
tree_gini = DecisionTreeClassifier(criterion="gini", random_state=42)
tree_entropy = DecisionTreeClassifier(criterion="entropy", random_state=42)

tree_gini.fit(X_train, y_train)
tree_entropy.fit(X_train, y_train)


## ۴. تنظیم پارامترهای مدل برای بهینه‌سازی

در این بخش با استفاده از `GridSearchCV` پارامترهای مدل را تنظیم می‌کنیم تا بهینه‌ترین مدل با دقت بالا به دست آید. این شامل محدودیت عمق درخت و حداقل تعداد نمونه‌های برگ است.

In [ ]:

# تنظیم پارامترهای مدل برای بهینه‌سازی
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_leaf': [1, 5, 10]
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=10)
grid_search.fit(X_train, y_train)
best_tree = grid_search.best_estimator_


## ۵. ارزیابی مدل‌ها و محاسبه دقت و ماتریس درهم‌ریختگی

با استفاده از مدل‌های ایجاد شده و مدل بهینه، دقت مدل‌ها را محاسبه می‌کنیم و ماتریس درهم‌ریختگی مدل بهینه را نمایش می‌دهیم.

In [ ]:

# ارزیابی مدل‌ها و محاسبه دقت و ماتریس درهم‌ریختگی
y_pred_gini = tree_gini.predict(X_test)
y_pred_entropy = tree_entropy.predict(X_test)
y_pred_best = best_tree.predict(X_test)

print("دقت مدل Gini:", accuracy_score(y_test, y_pred_gini))
print("دقت مدل Entropy:", accuracy_score(y_test, y_pred_entropy))
print("دقت مدل بهینه:", accuracy_score(y_test, y_pred_best))

# ماتریس درهم‌ریختگی
conf_matrix = confusion_matrix(y_test, y_pred_best)
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d")
plt.title("ماتریس درهم‌ریختگی مدل بهینه")
plt.show()


## ۶. گزارش دسته‌بندی

گزارش دسته‌بندی شامل دقت، بازیابی و نمره F1 برای مدل بهینه است.

In [ ]:

# گزارش دسته‌بندی
print(classification_report(y_test, y_pred_best))


## ۷. پیش‌بینی متغیر پاسخ برای داده‌های آزمون با استفاده از Cross-Validation

در این مرحله با استفاده از `Cross-Validation` دقت میانگین مدل بهینه را به دست می‌آوریم و نتایج Hold-out و Cross-Validation را مقایسه می‌کنیم.

In [ ]:

# پیش‌بینی متغیر پاسخ برای داده‌های آزمون با استفاده از Cross-Validation
cross_val_scores = cross_val_score(best_tree, X, y, cv=10)
print("دقت 10-Fold Cross-Validation:", cross_val_scores.mean())

# مقایسه نتایج Hold-Out و Cross-Validation
print("دقت Hold-Out:", accuracy_score(y_test, y_pred_best))
print("میانگین دقت Cross-Validation:", cross_val_scores.mean())


## ۸. نمایش درخت تصمیم بهینه

در نهایت، درخت تصمیم بهینه را به صورت گرافیکی نمایش می‌دهیم تا دید بهتری از مدل داشته باشیم.

In [ ]:

# نمایش درخت تصمیم بهینه
plt.figure(figsize=(20, 10))
plot_tree(best_tree, filled=True, feature_names=X.columns, class_names=best_tree.classes_)
plt.title("درخت تصمیم بهینه")
plt.show()
